<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/chatbot_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carichiamo il corpus di testo

In [0]:
import urllib.request, json 

with urllib.request.urlopen("https://raw.githubusercontent.com/ProfAI/nlp00/master/8%20-%20Chatbot/data/corpus.json") as url:
    data = url.read().decode()


In [2]:
corpus = json.loads(data)
corpus["intents"][0]

{'context_set': '',
 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
 'responses': ['Hello, thanks for visiting',
  'Good to see you again',
  'Hi there, how can I help?'],
 'tag': 'greeting'}

### Preprocessiamo i dati

In [7]:
import spacy
!python -m spacy download it_core_news_sm

    100% |████████████████████████████████| 36.5MB 116.1MB/s 
  Running setup.py install for it-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



In [24]:
nlp = spacy.load("en_core_web_sm")

dictionary = set({})
tags = []

docs = []


for intent in corpus["intents"]:
  
  for pattern in intent["patterns"]:
    
    pattern = pattern.lower()
    tokens = nlp(pattern)
    doc = ""
    
    for token in tokens:
      if(not token.is_punct and not token.is_stop):
        doc+=" "+token.lemma_
        dictionary.add(token.lemma_)
        
    if(len(doc)>0):
      docs.append(doc.rstrip()) 
      tags.append(intent["tag"])
  
print("Lunghezza del dizionario: %d" % len(dictionary))
print(docs)
print(tags)

Lunghezza del dizionario: 27
[' hi', ' hello', ' good day', ' bye', ' later', ' goodbye', ' thank', ' thank', ' be helpful', ' hour open', ' hour', ' open', ' like buy flower', ' want buy flower', ' flower', ' kind flower', ' credit card', ' credit card', ' accept mastercard', ' cash', ' rise flower', ' rose', ' rise', ' tulip flower', ' tulip', ' tulip', ' daisy flower', ' daisy', ' open today', ' open today', ' hour today']
['greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'hours', 'hours', 'hours', 'flowers', 'flowers', 'flowers', 'flowers', 'payments', 'payments', 'payments', 'payments', 'rose', 'rose', 'rose', 'tulip', 'tulip', 'tulip', 'daisy', 'daisy', 'opentoday', 'opentoday', 'opentoday']


### Bag of Words

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
X = bow.fit_transform(docs)
X.shape

(31, 27)

In [34]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(tags)
y[:5]

array([3, 3, 3, 2, 2])

In [35]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(y.reshape(-1, 1))
y.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(31, 10)

In [0]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation="relu", input_dim=X.shape[1]))
model.add(Dense(8, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [47]:
model.fit(X,y, epochs=500, batch_size=8)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
31/31 [==============================] - 1s 28ms/step - loss: 2.3048 - acc: 0.0968
Epoch 2/500
31/31 [==============================] - 0s 576us/step - loss: 2.3004 - acc: 0.1290
Epoch 3/500
31/31 [==============================] - 0s 555us/step - loss: 2.2965 - acc: 0.1290
Epoch 4/500
31/31 [==============================] - 0s 528us/step - loss: 2.2929 - acc: 0.1290
Epoch 5/500
31/31 [==============================] - 0s 588us/step - loss: 2.2893 - acc: 0.1290
Epoch 6/500
31/31 [==============================] - 0s 556us/step - loss: 2.2852 - acc: 0.1290
Epoch 7/500
31/31 [==============================] - 0s 504us/step - loss: 2.2813 - acc: 0.1290
Epoch 8/500
31/31 [==============================] - 0s 531us/step - loss: 2.2773 - acc: 0.1290
Epoch 9/500
31/31 [==============================] - 0s 562us/step - loss: 2.2732 - acc: 0.1290
Epoch 10/500
31/31 [==============================] - 0s 560us/step - loss: 2.2693 - acc:

### Testiamo il chatbot

In [0]:
def preprocess(sentence):
  
  tokens = nlp(sentence)
  doc = ""
  
  for token in tokens:
    if(not token.is_punct and not token.is_stop):
      doc+=" "+token.lemma_
  
  x = bow.transform([doc])
  
  return x

In [0]:
def chatbot(sentence):
  
  x = preprocess(sentence)
  y = model.predict_classes(x)
  tag = le.inverse_transform(y)
  
  return get_response(tag)

In [0]:
from random import choice

def get_response(tag):
  
  for intent in corpus["intents"]:
    if(intent["tag"]==tag):
      return choice(intent["responses"])

In [0]:
while(sentence!="bye" and sentence!=""):
  sentence = input("Tu: ")
  response = chatbot(sentence)
  print("Chatbot: "+response)